In [1]:
#Import Dependencies
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime as dt

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import table, column
import datetime

In [16]:
#Read in Data from CSV
drug_data_csv = "Resources/drug_deaths.csv"
income_data_csv = "Resources/per_capita_personal_income.csv"
drug_data = pd.read_csv(drug_data_csv)
income_data = pd.read_csv(income_data_csv)
#drug_data.head()
income_data.head()

,GeoFips,GeoName,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,...,2016:Q4,2017:Q1,2017:Q2,2017:Q3,2017:Q4,2018:Q1,2018:Q2,2018:Q3,2018:Q4,2022:Q1
0,0,United States,13782978.7,13937105.4,13907136.3,14386163.6,14010895.7,14150636.3,14240498.3,14354881.7,...,16310112.9,16550511.4,16736270.1,16925356.4,17167974.1,17383520.8,17573877.4,17795581.2,17971656.6,21237207.4


In [17]:
#Desired Data
new_drug_data = drug_data[['ID','Date', 'Age', 'ResidenceState', 'COD', 'Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone','Oxymorphone', 'Ethanol','Hydrocodone', 'Benzodiazepine', 'Methadone','Amphet', 'Tramad']].copy()
#new_drug_data.head()

new_income_data = income_data.iloc[:, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11 ,12 ,13, 14, 15, 16,17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,28,29]]
new_income_data.head()

,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,2014:Q1,2014:Q2,...,2016:Q3,2016:Q4,2017:Q1,2017:Q2,2017:Q3,2017:Q4,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,13782978.7,13937105.4,13907136.3,14386163.6,14010895.7,14150636.3,14240498.3,14354881.7,14616124.9,14868196.5,...,16148669,16310112.9,16550511.4,16736270.1,16925356.4,17167974.1,17383520.8,17573877.4,17795581.2,17971656.6


Data Cleaning: Drug Overdose Data

In [ ]:
#Extract only year from Date Column
new_drug_data['Date'] = pd.to_datetime(new_drug_data['Date'])
new_drug_data['Date'] = pd.DatetimeIndex(new_drug_data['Date']).year

In [ ]:
#Extract only year from Date Column
new_drug_data['Date'] = new_drug_data['Date'].fillna(0)
new_drug_data['Date'] = new_drug_data['Date'].astype(int)
new_drug_data = new_drug_data.set_index('Date')
new_drug_data.head()

In [ ]:
#Edit columns not int
new_drug_data["Fentanyl"].unique()
new_drug_data["Fentanyl"] = new_drug_data.apply(lambda row: int(row["Fentanyl"][:1]), axis=1)

In [ ]:
#Combine data to be per year (2012 to 2018)
drug_overdose_temp = new_drug_data.groupby(by='Date').agg({'Fentanyl': 'sum',
                            'Heroin': 'sum',
                            'Cocaine': 'sum',
                            'Oxycodone': 'sum',
                            'Oxymorphone': 'sum',
                            'Ethanol' : 'sum',
                            'Hydrocodone' : 'sum',
                            'Benzodiazepine': 'sum',
                            'Methadone': 'sum',
                            'Amphet' : 'sum',
                            'Tramad' : 'sum'}).reset_index()


In [ ]:
drug_overdose = drug_overdose_temp.drop(drug_overdose_temp.index[0])
drug_overdose

Data Cleaning: Personal Income Per Capita

In [25]:
#Concat Income Data
income_2012 = pd.concat([new_income_data['2012:Q1'], new_income_data['2012:Q2'],new_income_data['2012:Q3'],new_income_data['2012:Q4']],                  
                      ignore_index = True,
                      sort = False)

income_2013 = pd.concat([new_income_data['2013:Q1'], new_income_data['2013:Q2'],new_income_data['2013:Q3'],new_income_data['2013:Q4']],                  
                      ignore_index = True,
                      sort = False)

income_2014 = pd.concat([new_income_data['2014:Q1'], new_income_data['2014:Q2'],new_income_data['2014:Q3'],new_income_data['2014:Q4']],                  
                      ignore_index = True,
                      sort = False)

income_2015 = pd.concat([new_income_data['2015:Q1'], new_income_data['2015:Q2'],new_income_data['2015:Q3'],new_income_data['2015:Q4']],                  
                      ignore_index = True,
                      sort = False)

income_2016 = pd.concat([new_income_data['2016:Q1'], new_income_data['2016:Q2'],new_income_data['2016:Q3'],new_income_data['2016:Q4']],                  
                      ignore_index = True,
                      sort = False)

income_2017 = pd.concat([new_income_data['2017:Q1'], new_income_data['2017:Q2'],new_income_data['2017:Q3'],new_income_data['2017:Q4']],                  
                      ignore_index = True,
                      sort = False)

income_2018 = pd.concat([new_income_data['2018:Q1'], new_income_data['2018:Q2'],new_income_data['2018:Q3'],new_income_data['2018:Q4']],                  
                      ignore_index = True,
                      sort = False)

In [22]:
income_2012

0    13782978.7
1    13937105.4
2    13907136.3
3    14386163.6
dtype: float64

In [26]:
#Average income per year
average_2012 = income_2012.mean()
average_2013 = income_2013.mean()
average_2014 = income_2014.mean()
average_2015 = income_2015.mean()
average_2016 = income_2016.mean()
average_2017 = income_2017.mean()
average_2018 = income_2018.mean()

In [35]:
#Create new DF
yearly_income_USA = {
    'Income 2012':{average_2012},
    'Income 2013':{average_2013},
    'Income 2014':{average_2014},
    'Income 2015':{average_2015},
    'Income 2016':{average_2016},
    'Income 2017':{average_2017},
    'Income 2018':{average_2018}}

yearly_income_USA

#Note: may have to change this format

{'Income 2012': {14003346.000000002},
 'Income 2013': {14189228.0},
 'Income 2014': {14969526.975},
 'Income 2015': {15681233.0},
 'Income 2016': {16092713.025},
 'Income 2017': {16845028.0},
 'Income 2018': {17681159.0}}

Connect to Local Database 

In [ ]:
#Connect to DB
protocol = 'postgresql'
username = 'postgres'
password = ''
host = 'localhost'
port = 5433
database_name = 'drug_overdose_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
#Check for tables
engine.table_names()

In [ ]:
#engine.table_names()